In [2]:
import os

import polars as pl
import numpy as np

from glob import glob
from tqdm import tqdm

In [3]:
ls_path = sorted(glob("../valid_pairs/*.rm_dup_pairs.allValidPairs"))
ls_path

[]

In [3]:
ls_row_counts = []

for path in tqdm(ls_path):
    row_count = (
        pl.read_csv(
            path,
            has_header=False,
            sep='\t',
            n_threads=os.cpu_count() - 1,
            # n_rows=1000
        )
        .shape[0]
    )
    ls_row_counts.append(row_count)
ls_row_counts

100%|█████████████████████████████████████████████████████████████████████| 9/9 [20:44<00:00, 138.23s/it]


[137086054,
 278903904,
 314471781,
 587565817,
 607425712,
 612839424,
 580538160,
 174509415,
 430033524]

In [22]:
len(ls_row_counts)

9

In [28]:
df_vps = pl.DataFrame(
    zip([i.split('/')[-1] for i in ls_path], ls_row_counts), 
    columns=['sample_name', 'valid_pairs']
)
df_vps

sample_name,valid_pairs
str,i64
"""293T_HiC_DddA1...",137086054
"""293T_HiC_DddA1...",278903904
"""293T_HiC_DddA1...",314471781
"""293T_HiC_DddAw...",587565817
"""293T_HiC_DddAw...",607425712
"""293T_HiC_GFP_r...",612839424
"""293T_HiC_GFP_r...",580538160
"""293T_HiC_GFP_r...",174509415
"""293T_HiC_Natur...",430033524


In [29]:
df_vps.write_csv('tables/down-sampling_valid-pairs_per_sample.csv')

In [60]:
sample_count_min = df_vps['valid_pairs'].min()
sample_count_min

137086054

In [57]:
random_seed = 123

In [58]:
path_out = '../valid_pairs_down_sampling'
os.makedirs(path_out, exist_ok=True)

In [59]:
for path in tqdm(ls_path):
    (
        pl.read_csv(
            path,
            has_header=False,
            sep='\t',
            n_threads=os.cpu_count() - 1,
            # n_rows=1000
        )
        .sample(n=sample_count_min, seed=random_seed)
        .write_csv(
            path.replace('../valid_pairs', path_out),
            has_header=False,
            sep='\t'
        )
    )

100%|█████████████████████████████████████████████████████████████████████| 9/9 [22:04<00:00, 147.14s/it]
